In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")
import time

In [34]:
#options = EdgeOptions()
driver = webdriver.Edge()
driver.maximize_window()
data = pd.DataFrame()

driver.get("https://www.coindesk.com/markets/")

count = 1

for a in range(1,400):
     # Output to terminal repetitively over the same line using '\r'.
    driver.execute_script("window.scrollTo(10, document.body.scrollHeight);")
    for i in range(1,10):
        #sys.stdout.write("\rItem: {0}".format(count))
        #sys.stdout.flush()
        sys.stdout.write("\rRead: {0}".format(i))
        sys.stdout.flush()
        time.sleep(0.5)
        try:
            header = driver.find_element(By.XPATH, value="//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[2]/div[{0}]/div[2]/div[{1}]/div/div[2]/h5/a".format(a,i))
            desc = driver.find_element(By.XPATH, value="//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[2]/div[{0}]/div[2]/div[{1}]/div/div[2]/div[3]/p/span".format(a,i))
            news_date = driver.find_element(By.XPATH, value="//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[2]/div[{0}]/div[2]/div[{1}]/div/div[2]/div[5]/div[3]/div[1]/div[2]/span".format(a,i)) 
            
            record = pd.DataFrame({
                                    'header':header.text,
                                    'desc': desc.text,
                                    'date': news_date.text
                                }, index=[0])
            data = pd.concat([data, record],ignore_index=True, axis=0)
        except Exception as e:
            pass
            
    try:
        wait = WebDriverWait(driver, 10)
        more = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[3]/button")))
        more_articles = driver.find_element(By.XPATH, value="//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[3]/button")
        time.sleep(5)
        more_articles.click()
        time.sleep(1) 
        sys.stdout.write("\r          Button Progress: {0}".format(a))
        sys.stdout.flush()
    except Exception as e:
        pass
    
       
#driver.quit()
data.reset_index()
data.to_csv(r"Data/{0}_new_data.csv".format('Coindesk'), sep="\t")
print('Completed load all articles.')

          Button Progress: 399Completed load all articles.


### Use beutifulsoup to extract from html

If the script above not able to extract everything, save the html file and process it via beatiful soup as below method.

In [2]:
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import sys

In [3]:
with open(r'Data/CoinDesk/Markets _ CoinDesk.html','r', encoding="utf8") as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #define dataframe
    data = pd.DataFrame()

    for a in range(1,10):
        sys.stdout.write("\rSection: {0}".format(a))
        for i in range(1,10):
            try:
                header = dom.xpath("//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[2]/div[{0}]/div[2]/div[{1}]/div/div[2]/h5/a".format(a,i))
                desc = dom.xpath("//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[2]/div[{0}]/div[2]/div[{1}]/div/div[2]/div[3]/p/span".format(a,i))
                news_date = dom.xpath("//*[@id=\"fusion-app\"]/div/div[2]/div/main/div/div/div[2]/div[2]/div[{0}]/div[2]/div[{1}]/div/div[2]/div[5]/div[3]/div[1]/div[2]/span".format(a,i)) 
            
                #assign value into a record and append into data
                record = pd.DataFrame({
                            'header':header[0].text,
                            'desc': desc[0].text,
                            'date': news_date[0].text
                          }, index=[0])
                data = pd.concat([data, record],ignore_index=True, axis=0)
                sys.stdout.write("\r          Read: {0}".format(i))
                
            except:
                sys.stdout.write("\r                      No record for {0}".format(i))
        sys.stdout.flush()
    
    print(data.head())
    #driver.quit()
    data.reset_index()
    data.to_csv(r"Data/{0}_new_data.csv".format('Coindesk'), sep="\t")
    print('Completed load all articles.')

                      No record for 9                                              header  \
0  Facts or Feelings? Bitcoin Allocation Makes Se...   
1  Bitcoin Drops to Nearly $19K as Fed Renews Inf...   
2  First Mover Asia: How Traders Are Shorting Tet...   
3  Messari Research: DCG’s Barry Silbert Wins Fro...   
4  Genesis Faces ‘Hundreds of Millions’ in Losses...   

                                                desc          date  
0  If you run the numbers, bitcoin still belongs ...  Jun 30, 2022  
1  Central bank leaders warned Wednesday that inf...  Jun 30, 2022  
2  Hedge funds are increasingly betting against U...  Jun 30, 2022  
3  Messari’s Ryan Selkis says Grayscale's product...  Jun 30, 2022  
4  The DCG-owned trading colossus is said to have...  Jun 30, 2022  
Completed load all articles.


# Descriptive for CoinDesk News

In [5]:
import datetime as dt
from datetime import date
import numpy as np

news = pd.read_excel(r"Data/CoinDesk/coindesk_news.xlsx")

In [6]:
start = dt.datetime(2021,10,1)
end = dt.datetime(2022,6,30)

def filter_news_by_date(start, end, data):
    data.date = pd.to_datetime(data.date)
    data = data[(data['date'] >= start) & (data['date'] <= end)]
    data = data.set_index('date') 
    data = data.sort_index()
    data = data.reset_index()
    #data = data.set_index('date')
    return data

In [7]:
news = news.drop(['index'], axis=1)
coindesk_news_target_date = filter_news_by_date(start, end, news)
coindesk_news_target_date

,date,header,desc
0,2021-10-29,Curve Finance’s CRV Quietly Becomes a Top-Perf...,CRV was up 1.87% in the past 24 hours and near...
1,2021-10-29,"Play-to-Earn Squid Token Rockets 35,000% in 3 ...",The token buys entry to an online game inspire...
2,2021-10-29,Bitcoin Back Over $60K as El Salvador Buys 420...,"After a swoon in recent days, the largest cryp..."
3,2021-10-29,Market Wrap: Bitcoin Heads to $61K Ahead of Op...,Analysts expect short-term choppiness ahead of...
4,2021-10-30,Ether alcanza un récord de $4400 mientras shib...,La quema de monedas se refiere al proceso de r...
...,...,...,...
1117,2022-06-30,Genesis Faces ‘Hundreds of Millions’ in Losses...,The DCG-owned trading colossus is said to have...
1118,2022-06-30,Messari Research: DCG’s Barry Silbert Wins Fro...,Messari’s Ryan Selkis says Grayscale's product...
1119,2022-06-30,First Mover Asia: How Traders Are Shorting Tet...,Hedge funds are increasingly betting against U...
1120,2022-06-30,Bitcoin Drops to Nearly $19K as Fed Renews Inf...,Central bank leaders warned Wednesday that inf...
